In [1]:
import sys
sys.path.append('../modules/')
import pandas as pd
import matplotlib.pyplot as plt
import pulp
from mymodule_2 import *
from mymodule2_2 import *
from pulp_func_2 import *
df2023_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2023_result.columns = [column.rstrip(' ') for column in df2023_result.columns]
# print(df2023_result)

In [2]:
# 数据准备

merged_df = get_merged_df(df21, df22)
# print(merged_df)
total_yield_2023 = merged_df.groupby(['作物编号', '作物名称'])['单地块类型总产量/斤'].sum().reset_index()
total_yield_2023.columns = ['作物编号', '作物名称', '总产量/斤']
# print(total_yield_2023)
expected_sales_data_2023 = total_yield_2023.copy()
expected_sales_data_2023.columns = ['作物编号', '作物名称', '预期销售量/斤']
expected_sales_data = total_yield_2023.copy() # 初始值为2023年的数据
print(expected_sales_data)
expected_sales_data.columns = ['作物编号', '作物名称', '预期销售量/斤']

std_devs = [0.005, 0.005, 0.01, 0.01, 0.003, 0.003, 0.003, 0.003]

    作物编号 作物名称     总产量/斤
0      1   黄豆   57000.0
1      2   黑豆   21850.0
2      3   红豆   22400.0
3      4   绿豆   33040.0
4      5   爬豆    9875.0
5      6   小麦  170840.0
6      7   玉米  132750.0
7      8   谷子   71400.0
8      9   高粱   30000.0
9     10   黍子   12500.0
10    11   荞麦    1500.0
11    12   南瓜   35100.0
12    13   红薯   36000.0
13    14   莜麦   14000.0
14    15   大麦   10000.0
15    16   水稻   21000.0
16    17   豇豆   36480.0
17    18   刀豆   26880.0
18    19   芸豆    6480.0
19    20   土豆   30000.0
20    21  西红柿   36210.0
21    22   茄子   45360.0
22    23  菠菜      900.0
23    24   青椒    2610.0
24    25   菜花    3600.0
25    26   包菜    4050.0
26    27  油麦菜    4500.0
27    28  小青菜   35480.0
28    29   黄瓜   13050.0
29    30  生菜     2850.0
30    31   辣椒    1200.0
31    32  空心菜    3600.0
32    33  黄心菜    1800.0
33    34   芹菜    1800.0
34    35  大白菜  150000.0
35    36  白萝卜  100000.0
36    37  红萝卜   36000.0
37    38  榆黄菇    9000.0
38    39   香菇    7200.0
39    40  白灵菇   18000.0
40    41  羊肚菌   

In [3]:
fields = create_fields(df11)
# for field in fields:
#     print(field)
new_fields = create_new_fields(fields)
# for field in new_fields:
#     print(field)
crops = create_crops(df12, df22)
for crop in crops:
    print(crop)    

Crop(crop_id=1, crop_name=黄豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 400, 400], ['梯田', '单季', 380, 400], ['山坡地', '单季', 360, 400]], crop_price=3.25)
Crop(crop_id=2, crop_name=黑豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 500, 400], ['梯田', '单季', 475, 400], ['山坡地', '单季', 450, 400]], crop_price=7.5)
Crop(crop_id=3, crop_name=红豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 400, 350], ['梯田', '单季', 380, 350], ['山坡地', '单季', 360, 350]], crop_price=8.25)
Crop(crop_id=4, crop_name=绿豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 350, 350], ['梯田', '单季', 330, 350], ['山坡地', '单季', 315, 350]], crop_price=7.0)
Crop(crop_id=5, crop_name=爬豆, crop_type=粮食（豆类）, planting_fields=[['平旱地', '单季', 415, 350], ['梯田', '单季', 395, 350], ['山坡地', '单季', 375, 350]], crop_price=6.75)
Crop(crop_id=6, crop_name=小麦, crop_type=粮食, planting_fields=[['平旱地', '单季', 800, 450], ['梯田', '单季', 760, 450], ['山坡地', '单季', 720, 450]], crop_price=3.5)
Crop(crop_id=7, crop_name=玉米, crop_type=粮食, planting_fields=[['平旱

### 2 - 2024

In [4]:
# 更新数据
expected_sales_data = update_expected_sales_data(expected_sales_data, expected_sales_data_2023, std_devs)
# print(expected_sales_data)
crops = update_crops(crops, std_devs)
# for crop in crops:
#     print(crop)

In [5]:
variables, binary_variables, actual_sales, excess_yield = create_variables(new_fields, crops)
# 创建线性规划问题
prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)
# 定义目标函数的各个部分
k = 0.5
min_area_percent = 0.2
max_plots = 4
profit = define_objective_function(variables, actual_sales, excess_yield, crops, new_fields, k)
# 将目标函数添加到问题中
prob += profit
# print(profit)
prob = add_constraints2(prob, variables, binary_variables, actual_sales, excess_yield,
                        crops, new_fields, total_yield_2023, df2023_result, min_area_percent, max_plots)

In [6]:
# 求解问题
prob.solve()
print("Status:", pulp.LpStatus[prob.status])
update_new_fields(variables, new_fields)
# 打印结果
# for field in new_fields:
#     print(field.field_name, field.season, field.planted_crop)
df_template = pd.read_excel('../excels/附件3/template.xlsx')
output(df_template, 2024, new_fields, k)
# print(df_template)
objective_value = pulp.value(prob.objective)
print("Objective Function Value:", objective_value)

Status: Infeasible


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Objective Function Value: 8630943.656187767


### 2 - 2025-2030

In [7]:
for year in range(2025, 2031):
    k = 0.5
    min_area_percent = 0.2
    max_plots = 4
    list = [k, min_area_percent, max_plots, year]

    expected_sales_data = update_expected_sales_data(expected_sales_data, expected_sales_data_2023, std_devs)
    crops = update_crops(crops, std_devs)

    df_last1_result = pd.read_excel(f'../excels/附件3/{year - 1}_result_{k}.xlsx')
    df_last1_result.columns = [column.rstrip(' ') for column in df_last1_result.columns]
    if year - 2 == 2023:
        df_last2_result = pd.read_excel(f'../excels/附件3/2023_result.xlsx')
    else:    
        df_last2_result = pd.read_excel(f'../excels/附件3/{year - 2}_result_{k}.xlsx')
    df_last2_result.columns = [column.rstrip(' ') for column in df_last2_result.columns]
    optimize_planting_strategy(new_fields, crops, expected_sales_data, df_last1_result, df_last2_result, list)
    objective_value = pulp.value(prob.objective)
    print("Objective Function Value:", objective_value)

Status: Infeasible


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Objective Function Value: 8630943.656187767
Status: Infeasible


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Objective Function Value: 8630943.656187767
Status: Infeasible


...

Objective Function Value: 8630943.656187767
Status: Infeasible


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Objective Function Value: 8630943.656187767
Status: Infeasible


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Objective Function Value: 8630943.656187767
